In this notebook Madrid tables are ingested; data is explored; variables are renamed or created; exclusion criteria is applied; vitals, comorbidities, drugs and labs are appropriately transformed and cleaned; tables are joined; table 1 is produced; feature distribution is evaluated; dataset is split into training and internal validation; model is trained with ehr data, evaluated using 4 folds cross-validation and calibrated; model is externally evaluated with other datasets and feature importance is addressed. 

# Environment

In [ ]:
!pip install google-colab -q
!pip install shap -q
!pip install seaborn
!pip install tableone -q
!pip install sqldf

     |████████████████████████████████| 358kB 15.7MB/s 


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  87482      0 --:--:-- --:--:-- --:--:-- 87482
OK
97 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 97 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [ ]:
#Standard library imports
from google.colab import auth
auth.authenticate_user()

In [ ]:
!mkdir data
!gcsfuse structured_coviddsl_data data

!mkdir images
!gcsfuse new_cxr_30 images

!mkdir snuh_data
!gcsfuse snuh_covid snuh_data

2021/06/02 07:55:19.651615 Using mount point: /content/data
2021/06/02 07:55:19.661163 Opening GCS connection...
2021/06/02 07:55:20.067072 Mounting file system "structured_coviddsl_data"...
2021/06/02 07:55:20.099161 File system has been successfully mounted.
2021/06/02 07:55:20.282285 Using mount point: /content/images
2021/06/02 07:55:20.290933 Opening GCS connection...
2021/06/02 07:55:20.678561 Mounting file system "new_cxr_30"...
2021/06/02 07:55:20.679157 File system has been successfully mounted.
2021/06/02 07:55:20.916968 Using mount point: /content/snuh_data
2021/06/02 07:55:20.924073 Opening GCS connection...
2021/06/02 07:55:21.350838 Mounting file system "snuh_covid"...
2021/06/02 07:55:21.351438 File system has been successfully mounted.


# Libraries

In [ ]:
import os, sys, math
from tensorflow.python.lib.io import file_io
import glob
import warnings
from pandas_profiling import ProfileReport 

#Third party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tableone import TableOne
from scipy.stats import uniform, randint
from scipy.stats.mstats import winsorize
import seaborn as sns
import shap
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, precision_recall_curve, plot_precision_recall_curve, average_precision_score, brier_score_loss, roc_curve
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier

#Global configuration
pd.options.mode.chained_assignment = None
seed = 2020
np.random.seed(seed)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Tables ingestion

In [ ]:
patient = pd.read_csv('data/covid_dsl_v2/CDSL_01.csv')
patient = patient.set_index('PATIENT ID')

madrid_image_names = pd.read_csv('images/predictions/cxr_model/4fold_models_predictions_v2.csv')
madrid_image_names = pd.to_numeric(madrid_image_names['patient_id'])


vitals_inpatient = pd.read_csv('data/covid_dsl_v2/CDSL_02.csv',sep=';',encoding = "ISO-8859-1")
vitals_inpatient['PATIENT ID'] = vitals_inpatient['IDINGRESO']

dx_emerg = pd.read_csv('data/covid_dsl_v2/CDSL_03.csv',sep=';',encoding = "ISO-8859-1")
dx_emerg['PATIENT ID'] = dx_emerg['IDINGRESO']

lab = pd.read_csv('data/covid_dsl_v2/CDSL_06.csv',sep=';',encoding = "ISO-8859-1",error_bad_lines=False)
lab['PATIENT ID'] = lab['IDINGRESO']

b'Skipping line 28039: expected 8 fields, saw 12\nSkipping line 32182: expected 8 fields, saw 12\nSkipping line 44299: expected 8 fields, saw 12\n'
b'Skipping line 90790: expected 8 fields, saw 12\nSkipping line 105234: expected 8 fields, saw 12\nSkipping line 130735: expected 8 fields, saw 10\n'
b'Skipping line 136659: expected 8 fields, saw 10\nSkipping line 140398: expected 8 fields, saw 10\nSkipping line 141687: expected 8 fields, saw 12\nSkipping line 142745: expected 8 fields, saw 12\nSkipping line 170183: expected 8 fields, saw 12\n'
b'Skipping line 223336: expected 8 fields, saw 21\nSkipping line 237392: expected 8 fields, saw 12\n'
b'Skipping line 373889: expected 8 fields, saw 12\nSkipping line 382978: expected 8 fields, saw 10\n'
b'Skipping line 412136: expected 8 fields, saw 12\nSkipping line 419764: expected 8 fields, saw 12\nSkipping line 421226: expected 8 fields, saw 12\nSkipping line 426429: expected 8 fields, saw 11\nSkipping line 444865: expected 8 fields, saw 12\n'


In [ ]:
madrid_image_names.head

<bound method NDFrame.head of 0        738.0
1       1026.0
2       2433.0
3       2469.0
4       1117.0
         ...  
1845    1032.0
1846     399.0
1847    1827.0
1848    2358.0
1849     976.0
Name: patient_id, Length: 1850, dtype: float64>

# Initial patient table exploration

In [ ]:
#define timestamps
patient['admission_datetime'] = pd.to_datetime(patient['F_INGRESO/ADMISSION_DATE_URG/EMERG'].str.replace(' 00:00:00','') + ' ' + patient['HORA/TIME_ADMISION/ADMISSION_URG/EMERG'].str.replace('1899-12-30 ',''),format='%Y-%m-%d %H:%M:%S')
patient['discharge_datetime'] = pd.to_datetime(patient['F_ALTA/DISCHARGE_DATE_ING'],format='%Y-%m-%d')

#inspect the timestamps as a sanity check
print(f'First admission: {patient["admission_datetime"].min()}')
print(f'Last admission: {patient["admission_datetime"].max()}')
print(f'First discharge: {patient["discharge_datetime"].min()}')
print(f'Last discharge: {patient["discharge_datetime"].max()}')
print(f'Missing admission time: {patient["admission_datetime"].isnull().sum()}')
print(f'Missing discharge time: {patient["discharge_datetime"].isnull().sum()}')
print(f'Missing discharge disposition: {patient["MOTIVO_ALTA/DESTINY_DISCHARGE_ING"].isnull().sum()}')
print()

First admission: 2020-02-05 18:21:00
Last admission: 2020-06-10 19:49:00
First discharge: 2020-02-28 00:00:00
Last discharge: 2020-06-23 00:00:00
Missing admission time: 90
Missing discharge time: 3
Missing discharge disposition: 49



# Variables rename/creation

In [ ]:
patient_rename_dict = {'EDAD/AGE':'age',
                       'SEXO/SEX':'sex',
                       'admission_datetime':'admission_datetime',
                       #'DIAG_URG/EMERG':'ed_diagnosis',
                       #'RESPIRADOR/MECH.VENT.':'mechvent_flag',
                       'TEMP_PRIMERA/FIRST_URG/EMERG':'vitals_temp_ed_first',
                       #'TEMP_ULTIMA/LAST_URG/EMERG':'vitals_temp_ed_last',
                       'TA_MAX_PRIMERA/FIRST/EMERG_URG':'vitals_sbp_ed_first',
                       #'TA_MAX_ULTIMA/LAST_URGEMERG':'vitals_sbp_ed_last',
                       'TA_MIN_PRIMERA/FIRST_URG/EMERG':'vitals_dbp_ed_first',
                       #'TA_MIN_ULTIMA/LAST_URG/EMERG':'vitals_dbp_ed_last',
                       'FC/HR_PRIMERA/FIRST_URG/EMERG':'vitals_hr_ed_first',
                       #'FC/HR_ULTIMA/LAST_URG/EMERG':'vitals_hr_ed_last',
                       'SAT_02_PRIMERA/FIRST_URG/EMERG':'vitals_spo2_ed_first',
                       #'SAT_02_ULTIMA/LAST_URG/EMERG':'vitals_spo2_ed_last',
                       #'GLU_PRIMERA/FIRST_URG/EMERG':'vitals_glucose_ed_first',
                       #'GLU_ULTIMA/LAST_URG/EMERG':'vitals_glucose_ed_last',
                       'hospital_outcome':'hospital_outcome'}

# creating the variable hospital_outcome
patient['hospital_outcome'] = patient['MOTIVO_ALTA/DESTINY_DISCHARGE_ING'] == 'Fallecimiento'

patient = patient.rename(columns=patient_rename_dict)
patient = patient[list(patient_rename_dict.values())]
patient[list(patient_rename_dict.values())[:-1]] = patient[list(patient_rename_dict.values())[:-1]].replace({0:np.nan})

# Exclusion criteria

In [ ]:
print(':::Exclusion criteria:::')
print(f'Initial number of cases (covid_dsl_v2): {len(patient)}')

patient = patient[patient['age'] >= 16]
print(f'After excluding those with <16 Age: {len(patient)}')

patient = patient[patient["admission_datetime"].isnull() != True]
print(f'After excluding those with missing admission time: {len(patient)}')

patient = patient[patient['hospital_outcome'].isnull()==False]
print(f'After excluding those with missing hospital_outcome: {len(patient)}')

patient =  patient[patient.index.isin(madrid_image_names)]
print(f'After excluding those missing cxr: {len(patient)}')

print()

# Outcome distribution
print(':::Outcome distribution:::')
## inspect outcome distribution
print('Breakdown of hospital_outcome:')
print(patient['hospital_outcome'].value_counts())


:::Exclusion criteria:::
Initial number of cases (covid_dsl_v2): 2547
After excluding those with <16 Age: 2533
After excluding those with missing admission time: 2448
After excluding those with missing hospital_outcome: 2448
After excluding those missing cxr: 1628

:::Outcome distribution:::
Breakdown of hospital_outcome:
False    1439
True      189
Name: hospital_outcome, dtype: int64


# Vitals cleansing

In [ ]:
#ED diagnosis
#patient['ed_diagnosis'] = patient['ed_diagnosis'].replace({'DIFICULTAD RESPIRATORIA':'sx_breathing_difficulty',
#                                                             'FIEBRE':'sx_fever',
#                                                             'CUADRO CATARRAL':'sx_flu',
#                                                             'TOS':'sx_cough'}).apply(lambda row: row if 'sx' in str(row) else 'sx_others')

#Vital signs
features_list = [name for name in list(patient) if 'vitals' in name]

#Apply some clinical heuristics for valid ranges
limits = {'_sbp_':[20,240],
          '_hr_':[20,300],
          '_spo2_':[1,100],
          '_temp_':[30,45]}
for substr in limits.keys():
    for name in list(patient):
        if substr in name:
            patient[name][patient[name] < limits[substr][0]] = np.nan
            patient[name][patient[name] > limits[substr][1]] = np.nan 

#backfill the last ED vital sign if the first vital sign reading is not available
#for name in ['vitals_temp_ed_','vitals_sbp_ed_','vitals_dbp_ed_','vitals_hr_ed_','vitals_spo2_ed_']:
#    patient[name+'first'][patient[name+'first'].isnull()] = patient[name+'last']

#ventilator flag
#patient['mechvent_flag'] = patient['mechvent_flag'].apply(lambda row: 1 if row == 'SI' else 0)

keep = [name for name in patient.columns if 'last' not in name]
patient = patient.loc[:,keep]

patient.head()

,age,sex,admission_datetime,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,hospital_outcome
PATIENT ID,,,,,,,,,
1,85.0,MALE,2020-04-06 19:05:00,36.1,147.0,68.0,67.0,95.0,True
47,55.0,MALE,2020-03-16 11:53:00,36.0,139.0,87.0,75.0,98.0,False
48,70.0,FEMALE,2020-03-16 12:22:00,37.0,94.0,65.0,81.0,95.0,False
49,85.0,MALE,2020-03-16 12:39:00,35.8,132.0,59.0,NaN,89.0,False
50,39.0,MALE,2020-03-16 12:53:00,37.0,141.0,69.0,85.0,98.0,False


# Vital Signs

In [ ]:
# we are going to use the vitals from the ´patient´ table
# just in case, we are exploring the vitals_inpatient table
vitals_inpatient['timestamp'] = pd.to_datetime(vitals_inpatient['CONSTANTS_ING_DATE'] + ' ' + vitals_inpatient['CONSTANTS_ING_TIME'],format='%Y-%m-%d %H:%M')
vitals_inpatient_map = {'FC_HR_ING':'vitals_hr_inpatient',
                        'GLU_GLY_ING':'vitals_glucose_inpatient',
                        'SAT_02_ING':'vitals_spo2_inpatient',
                        'TA_MAX_ING':'vitals_sbp_inpatient',
                        'TA_MIN_ING':'vitals_dbp_inpatient',
                        'TEMP_ING':'vitals_temp_inpatient'}
vitals_inpatient = vitals_inpatient.rename(columns=vitals_inpatient_map)[['PATIENT ID','timestamp']+list(vitals_inpatient_map.values())]
vitals_inpatient['vitals_temp_inpatient'] = vitals_inpatient['vitals_temp_inpatient'].astype(str).str.replace(',','.').astype('float64')
vitals_inpatient = vitals_inpatient.applymap(lambda cell: np.nan if cell == 0 else cell)

vitals_24h = vitals_inpatient.merge(patient[['admission_datetime']],left_on='PATIENT ID',right_index=True)
vitals_24h = vitals_24h[vitals_24h['timestamp'] <= vitals_24h['admission_datetime'] + pd.Timedelta(days=1)]
hr_max = vitals_24h[['PATIENT ID','vitals_hr_inpatient']].groupby('PATIENT ID').max()
glucose_min = vitals_24h[['PATIENT ID','vitals_glucose_inpatient']].groupby('PATIENT ID').min() #highly missing
glucose_max = vitals_24h[['PATIENT ID','vitals_glucose_inpatient']].groupby('PATIENT ID').max() #highly missing
spo2_min = vitals_24h[['PATIENT ID','vitals_spo2_inpatient']].groupby('PATIENT ID').min() #this is all missing and is excluded
sbp_min = vitals_24h[['PATIENT ID','vitals_sbp_inpatient']].groupby('PATIENT ID').min()
dbp_min = vitals_24h[['PATIENT ID','vitals_dbp_inpatient']].groupby('PATIENT ID').min()
temp_min = vitals_24h[['PATIENT ID','vitals_temp_inpatient']].groupby('PATIENT ID').min()
temp_max = vitals_24h[['PATIENT ID','vitals_temp_inpatient']].groupby('PATIENT ID').max()
vitals_24h = pd.concat([hr_max,sbp_min,dbp_min,spo2_min,temp_min,temp_max],axis=1)
vitals_24h.head()

,vitals_hr_inpatient,vitals_sbp_inpatient,vitals_dbp_inpatient,vitals_spo2_inpatient,vitals_temp_inpatient,vitals_temp_inpatient
PATIENT ID,,,,,,
1,65.0,138.0,67.0,89.0,35.7,36.3
47,NaN,NaN,NaN,91.0,35.8,36.8
48,81.0,94.0,53.0,96.0,37.4,37.4
49,64.0,128.0,52.0,94.0,35.7,36.0
50,112.0,112.0,77.0,94.0,37.0,38.0


In [ ]:
vitals_inpatient

,PATIENT ID,timestamp,vitals_hr_inpatient,vitals_glucose_inpatient,vitals_spo2_inpatient,vitals_sbp_inpatient,vitals_dbp_inpatient,vitals_temp_inpatient
0,577,2020-02-06 23:36:00,NaN,NaN,98.0,NaN,NaN,NaN
1,585,2020-02-06 23:53:00,NaN,NaN,96.0,NaN,NaN,NaN
2,44,2020-02-22 10:32:00,NaN,NaN,99.0,NaN,NaN,NaN
3,586,2020-02-22 10:34:00,NaN,NaN,100.0,NaN,NaN,NaN
4,585,2020-02-22 11:56:00,NaN,NaN,96.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
128000,1590,2020-04-02 12:45:00,NaN,NaN,95.0,NaN,NaN,NaN
128001,122,2020-04-02 12:46:00,NaN,NaN,99.0,NaN,NaN,NaN
128002,122,2020-04-02 12:46:00,NaN,NaN,94.0,NaN,NaN,NaN
128003,2218,2020-04-02 12:46:00,NaN,NaN,83.0,NaN,NaN,NaN


# Comorbidities

In [ ]:
dx_inpat = pd.read_csv('data/covid_dsl_v2/CDSL_05.csv',sep=';',encoding = "ISO-8859-1")
dx_inpat['PATIENT ID'] = dx_inpat['IDINGRESO']

dx_combined = dx_emerg.merge(dx_inpat,left_on='PATIENT ID',right_on='PATIENT ID')

icd_10 = pd.get_dummies(dx_combined.iloc[:,dx_combined.columns.str.contains('DIA')],prefix='', prefix_sep='')
icd_10 = dx_combined[['PATIENT ID']].merge(icd_10,left_index=True,right_index=True)
icd_10 = icd_10.groupby(icd_10.columns, axis=1).max()

dx_dict = {'pmhx_diabetes':['E09','E10','E11'],
           'pmhx_hld':['E78'],
           'pmhx_htn':['I11','I13','I50'],
           'pmhx_ihd':['I25','Z95.1'],
           'pmhx_ckd':['N18','I12','I13','D63'],
           'pmhx_copd':['J44'],
           'pmhx_asthma':['J45'],
           'pmhx_activecancer':['C'],
           'pmhx_chronicliver':['K72','K76.6','K75.4','K74.6','K71.10','K74.3','K74.5','K70.30'],
           'pmhx_stroke':['I6'],
           'pmhx_chf':['I50'],
           'pmhx_dementia':['F01','F02','F03','G30','G31.0','G31.83']}

dx_onehot = icd_10[['PATIENT ID']]

for dx,code_list in zip(dx_dict.keys(),dx_dict.values()): 
    col_names = []
    for code in code_list:
        col_names += icd_10.columns[icd_10.columns.str.startswith(code)].tolist()
    dx_onehot[dx] = icd_10[col_names].apply(lambda row: np.max(row),axis=1) 
    
dx_onehot = dx_onehot.set_index('PATIENT ID')
dx_onehot.head()

,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia
PATIENT ID,,,,,,,,,,,,
144,1,0,0,0,0,0,0,1,0,0,0,0
145,0,0,0,0,0,0,0,0,0,0,0,0
146,0,0,0,0,0,0,0,0,0,0,0,0
147,0,0,0,0,0,0,0,0,0,0,0,0
148,0,1,0,0,0,0,0,0,0,0,0,0


# Drug

In [ ]:
# drugs variables did not improve the mdoel performance therefore were not inclued
drug = pd.read_csv('data/covid_dsl_v2/CDSL_04.csv',sep=';',encoding = "ISO-8859-1")
drug['timestamp'] = pd.to_datetime(drug['DRUG_START_DATE'],format='%Y-%m-%d')
drug['PATIENT ID'] = drug['IDINGRESO']
drug = drug.merge(patient[['admission_datetime']],left_on='PATIENT ID',right_index=True)
drug = drug[drug['timestamp'] <= drug['admission_datetime'] + pd.Timedelta(days=1)]

drug_dict = {'DOLQUINE comp 200 mg':'drug_hydroxychloroquine',
             'HIBOR jer 3.500 UI/ 0,2 mL':'drug_bemiparin_sodium',
             'AZITROMICINA comp 500 mg':'drug_azithromycin',
             'KALETRA/ALUVIA comp (200+50) mg':'drug_lopinavir_ritonavir',
             'METILPREDNISOLONA vial 40 mg':'drug_methylprednisolone',
             'HIBOR jer 7.500 UI/0,3 mL':'drug_bemiparin_sodium',
             'HIBOR jer 2.500 UI/0,2 mL':'drug_bemiparin_sodium',         
             'ALUVIA comp (200+50) mg':'drug_lopinavir_ritonavir',           
             'METILPREDNISOLONA vial 20 mg':'drug_methylprednisolone',
             'ROACTEMRA vial 200 mg/10 mL':'drug_tocilizumab',
             'DEXAMETASONA amp 4 mg/1 mL':'drug_dexamethasone',
             #Vasopressors
             'NORADRENALINA amp 10 mg/10 mL':'drug_vasopressor',
             'ADRENALINA amp 1 mg/1 mL':'drug_vasopressor',
             'EFEDRINA amp 50 mg/5 mL':'drug_vasopressor',
             'DOBUTAMINA amp 250 mg/20 mL':'drug_vasopressor',
             'DOPAMINA amp 200 mg/5 mL':'drug_vasopressor',
             #Intubation related drugs
             'PROPOFOL LIPURO vial 1% 50 mL':'drug_intubation',
             'PROPOFOL LIPURO amp 1% 20 mL':'drug_intubation',
             'PROPOFOL LIPURO vial 1% 100 mL':'drug_intubation',
             'FENTANEST amp 0,15 mg/3 mL':'drug_intubation',
             'REMIFENTANILO vial 5 mg':'drug_intubation',
             'PROPOFOL LIPOMED vial 2% 50 mL':'drug_intubation',
             'PROPOFOL LIPOMED amp 1% 20 mL':'drug_intubation',
             'CISATRACURIO vial  150 mg/30 mL':'drug_intubation',
             'DEXMEDETOMIDINA vial 1000 mcg /10 mL':'drug_intubation',
             'CISATRACURIO amp 10 mg/5 mL':'drug_intubation',
             'CISATRACURIO amp 20 mg/10 mL':'drug_intubation',
             'DEXMEDETOMIDINA vial 400 mcg/4 mL':'drug_intubation',
             'CISATRACURIO vial 20 mg/10 mL':'drug_intubation'}

#exclude: oxygen tank, disinfectant, barrier cream

new_drug_names = list(set(list(drug_dict.values())))
drug = drug.merge(pd.get_dummies(drug['DRUG_COMERCIAL_NAME']).rename(columns=drug_dict)[new_drug_names],left_index=True,right_index=True)
drug = drug.groupby(drug.columns, axis=1).sum()
drug = drug[['PATIENT ID']+new_drug_names].groupby(['PATIENT ID']).sum().applymap(lambda cell: 1 if cell >= 1 else 0)
drug.head()

,drug_dexamethasone,drug_tocilizumab,drug_hydroxychloroquine,drug_azithromycin,drug_lopinavir_ritonavir,drug_intubation,drug_vasopressor,drug_methylprednisolone,drug_bemiparin_sodium
PATIENT ID,,,,,,,,,
1,0,0,1,1,1,0,0,0,1
47,0,0,1,0,1,0,0,0,0
48,0,0,1,0,1,0,0,0,1
49,0,0,0,0,0,0,0,0,1
50,0,0,1,0,1,0,0,0,1


# Lab

In [ ]:
lab['timestamp'] = pd.to_datetime(lab['LAB_DATE'] +' '+ lab['TIME_LAB'],format='%d-%m-%Y %H:%M')

lab_map = {'NEU% -- Neutrófilos %':'lab_neutrophil_percentage',
           'HGB -- Hemoglobina':'lab_hemoglobin',
           'HCM -- Hemoglobina Corpuscular Media':'lab_mch',
           'ADW -- Coeficiente de anisocitosis':'lab_rdw',
           'NEU -- Neutrófilos':'lab_neutrophil',
           'VCM -- Volumen Corpuscular Medio':'lab_mcv',
           'LIN% -- Linfocitos %':'lab_lymphocyte_percentage',
           'HEM -- Hematíes':'lab_rbc',
           'LEUC -- Leucocitos':'lab_leukocyte',
           'HCTO -- Hematocrito':'lab_hct',
           'LIN -- Linfocitos':'lab_lymphocyte',
           #'CHCM -- Conc. Hemoglobina Corpuscular Media':'lab_mchc', --dervied from hb and rbc
           'PLAQ -- Recuento de plaquetas':'lab_platelet',
           'VPM -- Volumen plaquetar medio':'lab_mean_platelet_volume',
           'NA -- SODIO':'lab_sodium',
           'K -- POTASIO':'lab_potassium',
           #'EOS% -- Eosinófilos %':'lab_eosinophil_percentage',
           #'EOS -- Eosinófilos':'lab_eosinophil',
           #'BAS -- Basófilos':'lab_basophil',
           #'MONO% -- Monocitos %':'lab_monocyte_percentage',
           #'MONO -- Monocitos':'lab_monocyte',
           #'BAS% -- Basófilos %':'lab_basophil_percentage',
           'CREA -- CREATININA':'lab_creatinine',
           'U -- UREA':'lab_urea',
           'PCR -- PROTEINA C REACTIVA':'lab_crp',
           'GLU -- GLUCOSA':'lab_glucose',
           'GOT -- GOT (AST)':'lab_ast',
           'LDH -- LDH':'lab_ldh',
           'GPT -- GPT (ALT)':'lab_alt',
           'DD -- DIMERO D':'lab_ddimer',
           'INR -- INR':'lab_inr',
           #'TP -- TIEMPO DE PROTROMBINA':'lab_pt', --we will use INR instead
           'AP -- ACTIVIDAD DE PROTROMBINA':'lab_prothrombin_activity',
           'GGT -- GGT (GAMMA GLUTAMIL TRANSPEPTIDASA)':'lab_ggt',
           'APTT -- TIEMPO DE CEFALINA (APTT)':'lab_aptt',
           'PH -- pH':'lab_ph',
           'PO2 -- pO2':'lab_po2',
           'PCO2 -- pCO2':'lab_pco2',
           'SO2C -- sO2c (Saturación de oxígeno)':'lab_o2sats_arterial',
           'TCO2 -- tCO2(B)c':'lab_tco2',
           'BE(b) -- BE(b)':'lab_baseexcess',
           'BEecf -- BEecf':'lab_baseexcess_ecf',
           'HCO3 -- HCO3-':'lab_bicarb',
           'BT -- BILIRRUBINA TOTAL                                                               ':'lab_tbil',
           'LAC -- LACTATO':'lab_lactate',
           'CK -- CK (CREATINQUINASA)':'lab_ck',
           'CA++ -- Ca++ Gasometria':'lab_ionised_calcium_abg',
           'FA -- FOSFATASA ALCALINA':'lab_alp',
           'FIB -- FIBRINÓGENO':'lab_fibrinogen',
           'MG -- MAGNESIO':'lab_mg',
           'FOS -- FOSFORO':'lab_phosphate',
           'CL -- CLORO':'lab_chloride',
           'FER -- FERRITINA':'lab_ferritin',
           'BD -- BILIRRUBINA DIRECTA':'lab_bilirubin',
           #'G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO':'lab_coronavirus_pcr_exuded',
           'NA+P -- NA+':'lab_sodium',
           'K+P -- K+':'lab_potassium',
           'AMI -- AMILASA':'lab_amylase',
           'CA -- CALCIO                                                                          ':'lab_calcium',
           'ALB -- ALBUMINA':'lab_albumin',
           'HCO3V -- HCO3-':'lab_bicarb_venous',
           'BEecfV -- BEecf':'lab_baseexcess_ecf_venous',
           'PCO2V -- pCO2':'lab_pco2_venous',
           'PO2V -- pO2':'lab_po2_venous',
           'BE(b)V -- BE (b)':'lab_baseexcess_venous',
           'PHV -- pH':'lab_ph_venous',
           'SO2CV -- sO2c (Saturación de oxígeno)':'lab_o2sats_venous',
           #'CFLAG -- ALARMA HEMOGRAMA':'lab_hemoglobin_alarm',
           'TCO2V -- tCO2 (B)':'lab_tco2',
           'TROPO -- TROPONINA':'lab_troponin',
           'PROCAL -- PROCALCITONINA':'lab_procal',
           'PT -- PROTEINAS TOTALES':'lab_totalprotein',
           #'VIHAC -- VIH AC':'UNKNOWN',
           'AU -- ACIDO URICO':'lab_uricacid'}
           #'VSG -- VSG':'UNKNOWN',
           #'OBS_NULA2 -- Observaciones Bioquímica':'lab_biochemicalobservations',
           #'LEGIORI -- AG. LEGIONELA PNEUMOPHILA EN ORINA':'lab_urine_legionella_antigen',
           #'NEUMOORI -- AG NEUMOCOCO EN ORINA':'lab_urine_pneumoccal_antigen',
           #'AVISADOPOR -- Avisado por':'lab_advisedby',
           #'AVISADOA -- Avisado a':'lab_notifiedto',
           #'UROORS -- Urobilinógeno':'lab_urobilinogen',
           #'NITORS -- Nitritos':'lab_dipstick_nitrites',
           #'PTORS -- Proteinas':'lab_dipstick_proteins',
           #'ERIORS -- Eritrocitos':'lab_dipstick_erytocytes',
           #'LEUORS -- Leucocitos':'lab_dipstick_leukocytes',
           #'GLUORS -- Glucosa':'lab_dipstick_glucose',
           #'CETORS -- Cuerpos cetónicos':'lab_dipstick_ketones',
           #'BILORS -- Bilirrubina':'lab_dipstick_bil',
           #'DENORS -- Densidad':'lab_dipstick_density',
           #'PHORS -- pH':'lab_dipstick_ph',
           #'AVISADOPAR -- Parám. críticos':'lab_criticalparameter',
           #'TRIGLI -- TRIGLICERIDOS':'lab_triglycerides',
           #'VHBSAG -- AG. SUPERFICIE HEPATITIS B (AgHBs)':'lab_hbsag',
           #'VHBSAC -- AC. SUPERFICIE ANTI HEPATITIS B':'lab_antihbs',
           #'COREM -- AC. IGM ANTI HBC':'lab_antihbc_igm',
           #'VHCAC -- AC. ANTI HEPATITIS C':'lab_antihcv',
           #'OBS_CB -- Observaciones Coagulación':'lab_coagulation_observations',
           #'TSH -- TSH':'lab_tsh'}   

new_lab_names = list(set(list(lab_map.values())))
lab['ITEM_LAB'] = lab['ITEM_LAB'].replace(lab_map)

lab_24h = lab.merge(patient[['admission_datetime']],left_on='PATIENT ID',right_index=True)
lab_24h = lab_24h[lab_24h['timestamp'] <= lab_24h['admission_datetime'] + pd.Timedelta(days=1)]

lab_24h = lab_24h.pivot_table(values='VAL_RESULT', index=['PATIENT ID'],columns='ITEM_LAB',aggfunc='first')[new_lab_names]
lab_24h = lab_24h.applymap(lambda cell: cell if str(cell).replace('.','1').isdigit() else np.nan)
lab_24h = lab_24h.astype('float64')

#remove labs that have >50% missing values
remove = lab_24h.isnull().sum().index[(lab_24h.isnull().sum()/len(lab_24h) > 0.50)]
lab_24h = lab_24h.drop(columns=remove)
lab_24h.head()

ITEM_LAB,lab_ast,lab_rdw,lab_ddimer,lab_urea,lab_mcv,lab_creatinine,lab_alt,lab_crp,lab_potassium,lab_mch,lab_aptt,lab_lymphocyte,lab_ldh,lab_hemoglobin,lab_glucose,lab_neutrophil,lab_inr,lab_prothrombin_activity,lab_leukocyte,lab_rbc,lab_platelet,lab_mean_platelet_volume,lab_sodium,lab_neutrophil_percentage,lab_hct,lab_lymphocyte_percentage
PATIENT ID,,,,,,,,,,,,,,,,,,,,,,,,,,
1,46.0,14.6,6721.0,111.3,82.6,1.56,25.0,368.55,3.61,27.6,27.4,0.58,982.0,13.5,112.0,7.62,1.04,94.0,8.40,4.89,268.0,10.8,139.4,90.8,40.4,6.9
47,43.8,13.9,NaN,45.4,82.8,1.20,29.7,152.83,4.67,27.7,NaN,1.42,390.4,14.3,174.2,3.97,NaN,NaN,5.74,5.17,250.0,9.8,139.0,69.2,42.8,24.7
48,31.0,13.6,NaN,20.0,92.8,0.66,10.0,84.65,4.46,30.3,NaN,0.92,655.0,13.6,78.0,4.10,NaN,NaN,5.18,4.49,146.0,10.6,133.0,79.1,40.7,18.8
49,101.7,15.6,12785.0,99.4,100.9,1.28,49.1,71.39,5.99,30.7,28.0,0.50,913.4,7.1,130.1,6.21,1.31,68.0,7.57,2.31,157.0,11.8,138.1,82.0,23.3,6.6
50,27.0,12.6,NaN,25.0,90.2,1.18,17.0,26.98,4.75,30.2,35.6,0.99,217.0,16.0,89.0,2.25,1.30,66.0,3.47,5.28,121.0,10.6,144.0,71.0,47.6,27.4


In [ ]:
# get units
for item in lab_map.values():
    print(f'{item} - {lab["UD_RESULT"][lab["ITEM_LAB"]==item].unique()}')

lab_neutrophil_percentage - ['%' nan]
lab_hemoglobin - ['g/dL']
lab_mch - ['pg']
lab_rdw - ['%' nan]
lab_neutrophil - ['x10e3/µL' nan]
lab_mcv - ['fL']
lab_lymphocyte_percentage - ['%' nan]
lab_rbc - ['x10e6/µL']
lab_leukocyte - ['x10e3/µL']
lab_hct - ['%']
lab_lymphocyte - ['x10e3/µL' nan]
lab_platelet - ['x10e3/µL']
lab_mean_platelet_volume - ['fL' nan]
lab_sodium - ['mmol/L' nan]
lab_potassium - ['mmol/L' nan]
lab_creatinine - ['mg/dL' nan]
lab_urea - ['mg/dL' nan]
lab_crp - ['mg/L' nan]
lab_glucose - ['mg/dL' nan]
lab_ast - ['U/L' nan]
lab_ldh - ['U/L' nan]
lab_alt - ['U/L' nan]
lab_ddimer - ['ng/mL' nan]
lab_inr - [nan]
lab_prothrombin_activity - ['%' nan]
lab_ggt - ['U/L' nan]
lab_aptt - ['s' nan]
lab_ph - [nan]
lab_po2 - ['mmHg']
lab_pco2 - ['mmHg']
lab_o2sats_arterial - ['%']
lab_tco2 - ['mmol/L']
lab_baseexcess - ['mmol/L']
lab_baseexcess_ecf - ['mmol/L']
lab_bicarb - ['mmol/L']
lab_tbil - ['mg/dL' nan]
lab_lactate - ['mmol/L' nan]
lab_ck - ['U/L' nan]
lab_ionised_calcium_abg 

# Join all tables

In [ ]:
combined = patient.merge(pd.concat([dx_onehot,lab_24h],axis=1),how='left',left_index=True,right_index=True)

In [ ]:
#assert len(combined) == included_len, "Dataframe lengths differ after join!"

In [ ]:
combined.to_csv('hm_hospitales_covid_structured.csv')

In [ ]:
pd.options.display.max_seq_items  = 4000
print(combined.index)
combined.index.nunique()

Int64Index([   1,   47,   48,   49,   50,   51,   52,   53,   54,   55,   57,
              58,   59,   60,   62,   63,   64,   65,   66,   68,   70,   71,
              72,   74,   75,   76,   77,   78,   79,   81,   82,   83,   84,
              86,   87,   89,   90,   91,   93,   95,   96,   97,   99,  100,
             101,  103,  104,  106,  107,  108,  111,  112,  113,  115,  116,
             117,  118,  120,  123,  124,  125,  126,  128,  129,  130,  132,
             134,  135,  138,  141,  143,  146,  153,  154,  156,  158,  159,
             161,  164,  166,  167,  168,  169,  170,  173,  174,  176,  177,
             178,  179,  180,  181,  182,  184,  185,  186,  187,  188,  189,
             191,  192,  193,  195,  196,  197,  199,  200,  201,  202,  203,
             204,  205,  206,  207,  208,  209,  211,  212,  213,  214,  216,
             218,  220,  221,  224,  225,  227,  228,  230,  231,  232,  233,
             234,  242,  243,  247,  248,  249,  250,  251,  254

1628

# Table 1

In [ ]:
categorical = ['sex',
               'pmhx_activecancer', 'pmhx_asthma', 'pmhx_chf', 'pmhx_chronicliver', 'pmhx_ckd',
               'pmhx_copd', 'pmhx_dementia',
               'pmhx_diabetes', 'pmhx_hld','pmhx_htn', 'pmhx_ihd', 'pmhx_stroke','hospital_outcome']

print(TableOne(combined,columns=combined.drop(columns=['admission_datetime']).columns.tolist(),categorical=categorical,groupby='hospital_outcome',pval=True))

                                            Grouped by hospital_outcome                                                           
                                                                Missing          Overall            False             True P-Value
n                                                                                   1628             1439              189        
age, mean (SD)                                                        0      67.3 (15.8)      65.7 (15.7)      79.6 (10.0)  <0.001
sex, n (%)                           FEMALE                           0       648 (39.8)       594 (41.3)        54 (28.6)   0.001
                                     MALE                                     980 (60.2)       845 (58.7)       135 (71.4)        
vitals_temp_ed_first, mean (SD)                                     388       36.8 (0.8)       36.8 (0.8)       36.8 (0.9)   0.698
vitals_sbp_ed_first, mean (SD)                                      604     131.0 (

# Feature Distributions

In [ ]:
# plot feature distribution
df_y0 = combined[combined['hospital_outcome']==0]
df_y1 = combined[combined['hospital_outcome']==1]
continuous_vars = [column for column in combined.columns if column not in categorical+['admission_datetime']]

In [ ]:
#f, axes = plt.subplots(int(np.ceil(np.sqrt(len(continuous_vars)))), int(np.ceil(np.sqrt(len(continuous_vars)))), figsize=(20, 20), dpi=500)
#for ax, feature in zip(axes.flat, continuous_vars):
#    sns.distplot(df_y0[feature], ax=ax, label='Y=0')
#    sns.distplot(df_y1[feature], ax=ax, label='Y=1')
#    ax.legend()
#plt.show()

In [ ]:
for column in continuous_vars:
    print(f'{column} - min: {combined[column].min()}, max: {combined[column].max()}')

age - min: 18.0, max: 106.0
vitals_temp_ed_first - min: 33.2, max: 40.1
vitals_sbp_ed_first - min: 20.0, max: 200.0
vitals_dbp_ed_first - min: 31.0, max: 845.0
vitals_hr_ed_first - min: 21.0, max: 145.0
vitals_spo2_ed_first - min: 10.0, max: 99.0
lab_ast - min: 6.8, max: 323.2
lab_rdw - min: 7.6, max: 22.5
lab_ddimer - min: 17.0, max: 69987.0
lab_urea - min: 4.5, max: 545.6
lab_mcv - min: 53.1, max: 123.4
lab_creatinine - min: 0.35, max: 16.81
lab_alt - min: 1.8, max: 289.0
lab_crp - min: 0.37, max: 588.2
lab_potassium - min: 1.5, max: 6.39
lab_mch - min: 16.1, max: 43.1
lab_aptt - min: 17.8, max: 117.7
lab_lymphocyte - min: 0.04, max: 41.8
lab_ldh - min: 187.0, max: 6295.8
lab_hemoglobin - min: 5.3, max: 19.7
lab_glucose - min: 61.2, max: 550.9
lab_neutrophil - min: 0.04, max: 32.51
lab_inr - min: 0.92, max: 17.93
lab_prothrombin_activity - min: 4.0, max: 116.0
lab_leukocyte - min: 0.09, max: 49.4
lab_rbc - min: 2.31, max: 6.77
lab_platelet - min: 26.0, max: 1034.0
lab_mean_platelet_v

In [ ]:
combined_ = combined.reset_index()
combined_ = combined_.drop(columns='admission_datetime')

#dropping all drug columns, there are zero patients with 'vasopressor' and 'intubation' - this needs to be verified with data owners
combined_ = combined_[[name for name in combined_.columns if 'drug' not in name]]

def impute(series,method=None,missing_indicator=False): 
    """
    Wrapper function for sklearn's SimpleImputer

    Parameters
    ----------
    series: pd.Series
        a pd.Series to impute
    method: string
        passed to SimpleImputer's strategy parameter
    missing_indicator: bool
        logical flag to indicate if a missing value indicator column should be added

    Returns
    -------
    output_df: pd.DataFrame
        a pd.DataFrame containing the imputed series + missing indicator column
    """
    name = series.name
    si = SimpleImputer(strategy=method,add_indicator=missing_indicator)
    array = si.fit_transform(series.values.reshape(-1, 1))
    if missing_indicator:
        output_df = pd.DataFrame(array,columns=[name,name+'_missing'])
    else:
        output_df = pd.DataFrame(array,columns=[name])
    return output_df, si

def encode(series,drop_first=True):
    """
    Onehot encodes a categorical dataframe, dropping the first column

    Parameters
    ----------
    series: pd.Series
        a categorical pandas series
    drop_first: bool
        logical flag for whether the first category should be dropped

    Returns
    -------
    onehot_df: pd.DataFrame
        a pd.DataFrame containing the onehot encoded columns
    """
    onehot_df = pd.get_dummies(series,drop_first=drop_first)
    return onehot_df

def scale_center(series):
    """
    Wrapper function to scale and center a pd.Series using sklearn's Robust Scaler
    """
    rs = RobustScaler()
    array = rs.fit_transform(series.values.reshape(-1,1))
    array = array.flatten()
    series = pd.Series(array)
    return series, rs

any_missing = combined_.columns[combined_.isnull().sum() > 0].tolist()

imputer_list = []
scaler_list = []

for column in combined_.columns.tolist():
    #Categorical features
    if ('pmhx' in column) or ('drug' in column) or (column in ['ed_diagnosis','sex']):
        if column in any_missing:
            #Mode imputation
            imputed,imputer = impute(combined_[column],method='most_frequent')
            imputer_list += [(column,imputer)]
            combined_ = combined_.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if (len(combined_[column].value_counts()) > 2) or (combined_[column].dtype=='O'):
            #One hot encoding
            onehot_df = encode(combined_[column])
            combined_ = combined_.drop(columns=column).merge(onehot_df, left_index=True, right_index=True)
    #Numeric features - vital signs, laboratory values
    elif ('age' in column) or ('vitals' in column) or ('lab' in column):
        if column in any_missing:
            #Median imputation
            imputed,imputer = impute(combined_[column],method='median')
            imputer_list += [(column,imputer)]
            combined_ = combined_.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if 'spo2' not in column:
            #Winsorize to 1st and 99th percentile - excluding SpO2 which can normally take a value of 100%
            combined_[column] = winsorize(combined_[column],limits=(0.01,0.01))
        #Scale and center numeric columns
        combined_[column],scaler = scale_center(combined_[column])
        scaler_list += [(column,scaler)]
    
combined_ = combined_.set_index('PATIENT ID')

In [ ]:
combined_

,age,hospital_outcome,MALE,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia,lab_ast,lab_rdw,lab_ddimer,lab_urea,lab_mcv,lab_creatinine,lab_alt,lab_crp,lab_potassium,lab_mch,lab_aptt,lab_lymphocyte,lab_ldh,lab_hemoglobin,lab_glucose,lab_neutrophil,lab_inr,lab_prothrombin_activity,lab_leukocyte,lab_rbc,lab_platelet,lab_mean_platelet_volume,lab_sodium,lab_neutrophil_percentage,lab_hct,lab_lymphocyte_percentage
PATIENT ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.739130,True,1,-0.833333,1.545455,-1.166667,-1.400000,0.307692,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.820000,1.266667,21.708597,5.164725,-1.255814,2.576923,-0.140598,4.200530,-1.271739,-1.3125,-9.4,-0.955224,2.560338,-0.2500,-0.088578,1.335498,-3.0,3.000000,0.827723,0.365385,0.983165,0.6,0.727273,1.426230,-0.177778,-1.104167
47,-0.565217,False,1,-1.000000,0.818182,2.000000,-0.866667,1.230769,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.673333,0.800000,0.000000,0.778702,-1.209302,1.192308,0.189807,1.194398,1.032609,-1.2500,0.0,0.716418,-0.767932,0.2500,2.811189,-0.244589,0.0,0.000000,-0.225743,0.903846,0.740741,-0.4,0.606061,-0.344262,0.355556,0.750000
48,0.086957,False,0,0.666667,-3.272727,-1.666667,-0.466667,0.307692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.180000,0.600000,0.000000,-0.911814,1.116279,-0.884615,-1.195079,0.244287,0.576087,0.3750,0.0,-0.278607,0.720675,-0.1875,-1.673660,-0.188312,0.0,0.000000,-0.447525,-0.403846,-0.659933,0.4,-1.212121,0.467213,-0.111111,0.135417
49,0.739130,False,1,-1.333333,0.181818,-2.666667,0.000000,-1.538462,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,4.533333,1.933333,43.659729,4.372712,2.883721,1.500000,1.553603,0.059504,3.271739,0.6250,-8.2,-1.114428,2.174402,-3.1875,0.755245,0.725108,2.4,-1.333333,0.499010,-3.211538,-0.511785,1.6,0.333333,0.704918,-3.200000,-1.135417
50,-1.260870,False,1,0.666667,1.000000,-1.000000,-0.200000,1.230769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.446667,-0.066667,0.000000,-0.579035,0.511628,1.115385,-0.702988,-0.559365,1.206522,0.3125,7.0,-0.139303,-1.552180,1.3125,-1.160839,-0.989177,2.2,-1.666667,-1.124752,1.115385,-0.996633,0.4,2.121212,-0.196721,1.422222,1.031250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,-0.173913,False,1,-1.000000,-2.727273,1.500000,2.133333,0.923077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2565,1.043478,False,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.446667,0.333333,0.380090,0.000000,1.232558,-0.538462,-1.398946,-0.497213,0.380435,0.6875,-2.4,0.616915,0.000000,-0.9375,-1.174825,-0.764069,-0.2,-0.166667,-0.459406,-1.307692,0.781145,-0.9,0.151515,-1.606557,-0.822222,0.947917
2567,-0.652174,False,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [ ]:
combined_.to_csv('hm_hospitales_mc_std.csv')

In [ ]:
# plot feature distribution
df_y0 = combined_[combined_['hospital_outcome']==0]
df_y1 = combined_[combined_['hospital_outcome']==1]

In [ ]:
#f, axes = plt.subplots(int(np.ceil(np.sqrt(len(continuous_vars)))), int(np.ceil(np.sqrt(len(continuous_vars)))), figsize=(20, 20), dpi=500)
#for ax, feature in zip(axes.flat, continuous_vars):
#    sns.distplot(df_y0[feature], ax=ax, label='Y=0')
#    sns.distplot(df_y1[feature], ax=ax, label='Y=1')
#    ax.legend()
#plt.show()

# Final Predictors List

In [ ]:
list(combined_)

['age',
 'hospital_outcome',
 'MALE',
 'vitals_temp_ed_first',
 'vitals_sbp_ed_first',
 'vitals_dbp_ed_first',
 'vitals_hr_ed_first',
 'vitals_spo2_ed_first',
 'pmhx_diabetes',
 'pmhx_hld',
 'pmhx_htn',
 'pmhx_ihd',
 'pmhx_ckd',
 'pmhx_copd',
 'pmhx_asthma',
 'pmhx_activecancer',
 'pmhx_chronicliver',
 'pmhx_stroke',
 'pmhx_chf',
 'pmhx_dementia',
 'lab_crp',
 'lab_aptt',
 'lab_hemoglobin',
 'lab_inr',
 'lab_lymphocyte',
 'lab_ast',
 'lab_ldh',
 'lab_mean_platelet_volume',
 'lab_urea',
 'lab_glucose',
 'lab_lymphocyte_percentage',
 'lab_creatinine',
 'lab_sodium',
 'lab_mcv',
 'lab_neutrophil',
 'lab_rbc',
 'lab_ddimer',
 'lab_leukocyte',
 'lab_potassium',
 'lab_prothrombin_activity',
 'lab_platelet',
 'lab_alt',
 'lab_hct',
 'lab_mch',
 'lab_neutrophil_percentage',
 'lab_rdw']

# Train-test split

In [ ]:
X = combined_.drop(columns='hospital_outcome')
Y = combined_['hospital_outcome']

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.25,random_state=seed)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1836, 45)
(612, 45)
(1836,)
(612,)


In [ ]:
# exporting training and testing ids to csv
train_export = pd.DataFrame({'patient_id':y_train.index, 'hospital_outcome':y_train.values})
train_export.to_csv('train_export.csv', index=False)

test_export = pd.DataFrame({'patient_id':y_test.index, 'hospital_outcome':y_test.values})
test_export.to_csv('test_export.csv', index=False)

def copy_file_to_gcs(job_dir, file_path):
    with file_io.FileIO(file_path, mode='rb') as input_f:
        with file_io.FileIO(os.path.join(job_dir, file_path), mode='wb+') as output_f:
            output_f.write(input_f.read())

copy_file_to_gcs('gs://first_cxr/split/','train_export.csv')
copy_file_to_gcs('gs://first_cxr/split/','test_export.csv')       

# Model Selection

In [ ]:
from sklearn.metrics import roc_auc_score, recall_score, make_scorer

def clipped_auc_scorer(y_true,y_pred):
    """
    Calculates the AUROC score, but returns 0 if sensitivity of the positive class is < 0.7
    """
    auc_score = roc_auc_score(y_true, y_pred)
    if recall_score(y_true,y_pred) >= 0.7:
        return auc_score
    else:
        return 0

clipped_auc = make_scorer(clipped_auc_scorer)

In [ ]:
def model_selection(summary_dict,model_lst,param_dict,
                    x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test,
                    n_iter=None,k_fold=None,n_repeats=None):
    """
    A wrapper function for the model selection loop
    
    Parameters
    ----------
    summary_dict: dict
        An empty dictionary used to store results.
    model_lst: list
        A list of tuples containing ('model_name',model), models are sklearn estimators
    param_dict: dict
        A dictionary containing model parameter distributions - to be passed to RandomizedSearchCV
    x_train: array-like
        An array training set predictors
    y_train: array-like
        An array containing training set labels
    x_test: array-like
        An array containing test set predictors
    y_test: array-like
        An array containing test set labels
    n_iter: int
        Number of crossvalidation iterations - to be passed to RandomizedSearchCV. Defaults to n_iter parameter at top of script
    k_fold: int
        Number of crossvalidation folds - to be passed to RandomizedSearchCV. Defaults to k_fold parameter at top of script
    n_repeats: int
        Number of crossvalidation repeats - to be passed to RandomizedSearchCV. Defaults to n_repeats parameter at top of script
        
    Returns
    -------
    summary_dict: pandas.DataFrame
        A dataframe containing the best model object and associated crossvalidation results
    result_table: pandas.DataFrame
        A dataframe containing all model objects and associated crossvalidation results
    """
    iterations = n_iter
    
    #Full list of scoring metrics
    scoring = {'roc_auc':'roc_auc',
               'average_precision':'average_precision',
               'accuracy': 'accuracy',
               'f1_macro':'f1_macro',
               'clipped_auc':clipped_auc}

    #Create an empty list used to store the results
    result_list = []
    
    #Loop through the list of models
    for name, model in model_lst:

        #Define the cross-validation folds
        cv = RepeatedStratifiedKFold(n_splits=k_fold,n_repeats=n_repeats)
        
        #Set the optimizing metric
        refit_score = 'clipped_auc'
        
        #Set the estimator as the model currently being optimized
        estimator = model
        
        for sample_weight_method in ['unbalanced','balanced']:
            for method in ['sigmoid','none']:
                #Wrap the model into calibration loop
                if method != 'none':
                    clf = CalibratedClassifierCV(estimator,method=method)
                    params = param_dict
                else:
                    clf= estimator
                    new_dict = {}
                    for key in param_dict.keys():
                        subdict = {}
                        for nested_key in param_dict[key].keys():
                            new_key_name = nested_key.split('base_estimator__')[1]
                            subdict[new_key_name] = param_dict[key][nested_key]
                        new_dict[key] = subdict
                    params = new_dict

                #Create the RandomizedSearchCV object
                search = RandomizedSearchCV(clf,param_distributions=params.get(name),random_state=seed,cv=cv,n_iter=iterations,n_jobs=-1,
                                        scoring=scoring,refit=refit_score,verbose=False,return_train_score=False)

                #Begin the grid search process
                if sample_weight_method == 'balanced':
                    search.fit(x_train, y_train, sample_weight=weight_array) 
                else:
                    search.fit(x_train, y_train) 

                #Calculate some metrics on the full training dataset (purely for diagnostics)
                y_pred = search.best_estimator_.predict(x_train)

                print(f'Algorithm: {name}')
                print(f'Calibration method: {method}')
                print(f'Sample weight method: {sample_weight_method}')
                print(classification_report(y_true=y_train,y_pred=y_pred))
                print(f'CV score ({refit_score}): {search.best_score_}')
                print()
            
                #Append the results of the best model to results_list
                result_list.append((name+'_'+method+'_'+sample_weight_method,search,search.best_score_,search.cv_results_))
        
        ##End of loop
    
    #The following code tidies result_list in to a dataframe
    result_table = pd.DataFrame(result_list,columns=['name','model','scores','score_dict'])
    best_model_index = result_table['scores']==max(result_table['scores'])
    model_name = result_table['name'][best_model_index].values.tolist()[0]
    best_model = result_table['model'][best_model_index].values.tolist()[0]
    summary_dict['results'] = {'Model':model_name}
    metrics = ['mean_test_roc_auc','mean_test_average_precision','mean_test_accuracy']
    if hasattr(best_model,'best_score_'):
        best_score = best_model.best_score_ 
        for key in [key for key in best_model.cv_results_.keys() if key in metrics]:
            summary_dict['results'][key.split('mean_test_')[1]] = best_model.cv_results_[key][best_model.best_index_]
        summary_dict['results']['model obj'] = best_model.best_estimator_
    else:
        best_score = mean_cv_roc_auc
        summary_dict['results']['model obj'] = best_model
        for key in [key for key in best_search.cv_results_.keys() if key in metrics]:
            summary_dict['results'][key.split('mean_test_')[1]] = result_table['score_dict'][best_model_index].get(key)
    
    #Find the overall results
    print(f"Best Cross-Validation score: {best_score}")        
    
    return summary_dict, result_table

In [ ]:
#This cell runs the model selection loop 
summary_dict = {}

#Create model objects
logistic = SGDClassifier(loss='log',random_state=seed)
rf = RandomForestClassifier(random_state=seed)
gbm = GradientBoostingClassifier(random_state=seed)
xgbclf = XGBClassifier(random_state=seed,objective='binary:logistic')

classifier_list = [('xgb',xgbclf),('lr',logistic)] #,('gbm',gbm),('rf',rf),

#Calculate class weights
weight_array = compute_sample_weight(class_weight="balanced",y=y_train)

#Define the hyperparameter search space
params = {'lr':{'base_estimator__alpha':uniform(1e-5,1e-3),
                'base_estimator__penalty':['l1', 'l2', 'elasticnet'],
                'base_estimator__l1_ratio':uniform(0.01,0.30)},
          'rf':{'base_estimator__bootstrap': [True, False],
                'base_estimator__max_depth':randint(3,12),
                'base_estimator__max_features': ['auto', 'sqrt'],
                'base_estimator__min_samples_split':randint(2,12),
                'base_estimator__min_samples_leaf':randint(2,12),
                'base_estimator__n_estimators':randint(200, 1000)},
          'gbm':{'base_estimator__loss':['deviance','exponential'],
                 'base_estimator__learning_rate':uniform(0.003, 0.3),
                 'base_estimator__n_estimators':randint(200, 1000),
                 'base_estimator__subsample':uniform(0.5, 0.5),
                 'base_estimator__criterion':['friedman_mse','mse','mae'],
                 'base_estimator__min_samples_split':randint(2,12),
                 'base_estimator__min_samples_leaf':randint(2,12),
                 'base_estimator__max_depth':randint(3,12),
                 'base_estimator__max_features':['sqrt', 'log2']},
          'xgb':{'base_estimator__colsample_bytree': uniform(0.5,0.5),
                 'base_estimator__eta': (0.0001,0.1),
                 'base_estimator__max_depth': randint(3,12),
                 'base_estimator__min_child_weight': randint(3,12),
                 'base_estimator__subsample': uniform(0.5,0.5)}}

#Run the model selection loop
summary_dict, results = model_selection(summary_dict=summary_dict,
                                        model_lst=classifier_list,
                                        param_dict=params,
                                        n_iter=20,k_fold=5,n_repeats=1)

Algorithm: xgb
Calibration method: sigmoid
Sample weight method: unbalanced
              precision    recall  f1-score   support

       False       0.94      1.00      0.97      1576
        True       0.97      0.59      0.74       260

    accuracy                           0.94      1836
   macro avg       0.95      0.79      0.85      1836
weighted avg       0.94      0.94      0.93      1836

CV score (clipped_auc): 0.0

Algorithm: xgb
Calibration method: none
Sample weight method: unbalanced
              precision    recall  f1-score   support

       False       0.96      1.00      0.98      1576
        True       0.97      0.72      0.83       260

    accuracy                           0.96      1836
   macro avg       0.96      0.86      0.90      1836
weighted avg       0.96      0.96      0.95      1836

CV score (clipped_auc): 0.0

Algorithm: xgb
Calibration method: sigmoid
Sample weight method: balanced
              precision    recall  f1-score   support

       Fal

KeyboardInterrupt: ignored

# Evaluation

In [ ]:
cv_results = results.sort_values('scores',ascending=False)

clf_list = list(zip(cv_results['name'].iloc[0:4],cv_results['model'].iloc[0:4]))
plt.figure(figsize=(6,5),dpi=100)
plt.title('Receiver Operating Characteristic')

color_list = ['#581845','#900C3F','#C70039','#FF5733']
col = 0

from sklearn import metrics

for clf_name,clf in clf_list:
    clf = clf.best_estimator_
    y_prob = clf.predict_proba(x_test)
    prob = y_prob[:,1]
    fpr, tpr, threshold = roc_curve(y_test, prob)
    roc_auc = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, color_list[col], label = f'{clf_name} = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'k--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    col += 1
plt.show()

In [ ]:
for name,best_model in clf_list:
    best_model = best_model.best_estimator_
    y_score = best_model.predict_proba(x_test)[:,1]
    average_precision = average_precision_score(y_test, y_score)
    disp = plot_precision_recall_curve(best_model, x_test, y_test)
    disp.ax_.set_title(f'{name} 2-class Precision-Recall curve: '
                    'AP={0:0.2f}'.format(average_precision))

In [ ]:
from sklearn.metrics import roc_auc_score

model = results['model'][results['scores']==max(results['scores'])].iloc[0].best_estimator_

bootstrap_dict = {'F1':[],'AUPRC':[],'AUROC':[],'NPV':[],'PPV':[],'Sensitivity':[],'Specificity':[]}
threshold = 0.5
n_resamples = 1000
for i in range(n_resamples):
    idx = np.random.choice(range(len(x_test)), size=len(x_test), replace=True)
    prob_pos = model.predict_proba(x_test.iloc[idx,:])[:,1]
    y_hat = prob_pos >= threshold
    clf_report = classification_report(y_true=y_test.iloc[idx],y_pred=y_hat,output_dict=True)
    bootstrap_dict['F1'].append(clf_report['macro avg']['f1-score'])
    bootstrap_dict['AUPRC'].append(average_precision_score(y_true=y_test.iloc[idx], y_score=prob_pos,average='macro'))    
    bootstrap_dict['AUROC'].append(roc_auc_score(y_true=y_test.iloc[idx],y_score=prob_pos))
    bootstrap_dict['NPV'].append(clf_report['False']['precision'])
    bootstrap_dict['PPV'].append(clf_report['True']['precision'])
    bootstrap_dict['Sensitivity'].append(clf_report['True']['recall'])
    bootstrap_dict['Specificity'].append(clf_report['False']['recall'])

In [ ]:
y_hat = model.predict(x_test)
print(results['name'][results['scores']==max(results['scores'])].iloc[0])
print(classification_report(y_true=y_test,y_pred=y_hat))

for key in bootstrap_dict.keys():
    mean = np.round(np.mean(bootstrap_dict[key]),3)
    lower = np.round(np.quantile(bootstrap_dict[key],0.025),3)
    upper = np.round(np.quantile(bootstrap_dict[key],0.975),3)
    print(f'{key}: {mean} (95% CI {lower}-{upper})')

# Calibration

In [ ]:
import matplotlib.pyplot as plt

title_calibration='Calibration Plot'
print(results['name'][results['scores']==max(results['scores'])].iloc[0])
plt.figure(figsize=(10,10),dpi=100)
plt.subplot(2,2,4)
plt.title(title_calibration)
plt.ylim(0.,1.)
plt.xlim(0.,1.)
plt.ylabel("Fraction Of Positives")
plt.xlabel("Mean Predicted Value")
plt.plot([1, 0], [1, 0],"k--",linewidth=1)
y_score = model.predict_proba(x_test)[:,1]
fraction_of_positives, mean_predicted_value = calibration_curve(y_test,y_score,n_bins=10)
brier_score = brier_score_loss(y_test,y_score)
plt.plot(mean_predicted_value, fraction_of_positives, "rs-",markersize=1,label=f'Brier Score: {np.round(brier_score,3)}',color='indigo',linewidth=1)
plt.legend(loc="lower right")
plt.tight_layout()

# External Validation

## Hoboken

In [ ]:
external = pd.read_excel('Hoboken structured data.xlsx')

external.columns = [col.split(',')[0] for col in external.columns]

external['MALE'] = external['sex']

external_vars = ['age', 'hospital_outcome', 'MALE', 'vitals_temp_ed_first', 'vitals_sbp_ed_first',
       'vitals_dbp_ed_first', 'vitals_hr_ed_first', 'pmhx_diabetes',
       'pmhx_hld', 'pmhx_htn', 'pmhx_ihd', 'pmhx_ckd', 'pmhx_copd',
       'pmhx_asthma', 'pmhx_activecancer', 'pmhx_chronicliver', 'pmhx_stroke',
       'pmhx_chf', 'pmhx_dementia', 'lab_mcv', 'lab_mch', 'lab_rdw', 'lab_rbc',
       'lab_mean_platelet_volume', 'lab_hct', 'vitals_spo2_ed_first',
       'lab_ldh', 'lab_neutrophil_percentage', 'lab_aptt', 'lab_ddimer',
       'lab_inr', 'lab_glucose', 'lab_urea', 'lab_lymphocyte_percentage',
       'lab_ast', 'lab_neutrophil', 'lab_hemoglobin', 'lab_crp', 'lab_alt',
       'lab_creatinine', 'lab_platelet', 'lab_prothrombin_activity',
       'lab_leukocyte', 'lab_sodium', 'lab_lymphocyte', 'lab_potassium']

external = external[external_vars]

cat_list = ['hospital_outcome','pmhx_diabetes','pmhx_hld','pmhx_htn','pmhx_ihd','pmhx_ckd','pmhx_copd','pmhx_asthma','pmhx_activecancer','pmhx_chronicliver','pmhx_stroke','pmhx_chf','pmhx_dementia']
external[cat_list] = external[cat_list].fillna(0)

external['lab_ddimer'][external['lab_ddimer'] == '< 200'] = 200

In [ ]:
def impute_external(col_name,imputer_list,data):
    try:
        for name,imputer in imputer_list:
            if name == col_name:
                imputed_data = imputer.transform(data.to_numpy().reshape(-1, 1))
                imputed_data = imputed_data.ravel()
    except BaseException as e:
        print(f'Column: {col_name} encountered exception {e}')
    return pd.Series(imputed_data,name=col_name)

def scale_external(col_name,scaler_list,data):
    try:
        for name,scaler in scaler_list:
            if name == col_name:
                scaled_data = scaler.transform(data.to_numpy().reshape(-1, 1))
                scaled_data = scaled_data.ravel()
    except BaseException as e:
        print(f'Column: {col_name} encountered exception {e}')
    return pd.Series(scaled_data,name=col_name)

In [ ]:
any_missing = external.columns[external.isnull().sum() > 0].tolist()

for column in external.columns.tolist():
    #Categorical features
    if ('pmhx' in column) or ('drug' in column) or (column in ['ed_diagnosis','sex']):
        if column in any_missing:
            #Mode imputation
            imputed = impute(external[column],method='most_frequent')
            external = external.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if (len(external[column].value_counts()) > 2) or (external[column].dtype=='O'):
            #One hot encoding
            onehot_df = encode(external[column])
            external = external.drop(columns=column).merge(onehot_df, left_index=True, right_index=True)
    #Numeric features - vital signs, laboratory values
    elif ('age' in column) or ('vitals' in column) or ('lab' in column):
        if column in any_missing:
            #Median imputation
            imputed = impute_external(col_name=column,imputer_list=imputer_list,data=external[column])
            external = external.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if 'spo2' not in column:
            #Winsorize to 1st and 99th percentile - excluding SpO2 which can normally take a value of 100%
            external[column] = winsorize(external[column],limits=(0.01,0.01))
        #Scale and center numeric columns
        external[column] = scale_external(col_name=column,scaler_list=scaler_list,data=external[column])

In [ ]:
y_external = external['hospital_outcome']
x_external = external.drop('hospital_outcome',axis=1)
x_external = x_external[x_train.columns]

In [ ]:
%matplotlib inline
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'

In [ ]:
y_prob = clf.predict_proba(x_external)
prob = y_prob[:,1]
fpr, tpr, threshold = roc_curve(y_external, prob)
roc_auc = metrics.auc(fpr, tpr)
plt.plot(fpr, tpr, label = f'{clf_name} = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'k--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
ext_bootstrap_dict = {'F1':[],'AUPRC':[],'AUROC':[],'NPV':[],'PPV':[],'Sensitivity':[],'Specificity':[]}
threshold = 0.5
n_resamples = 1000
for i in range(n_resamples):
    idx = np.random.choice(range(len(x_external)), size=len(x_external), replace=True)
    prob_pos = model.predict_proba(x_external.iloc[idx,:])[:,1]
    y_hat = prob_pos >= threshold
    ext_clf_report = classification_report(y_true=y_external.iloc[idx],y_pred=y_hat,output_dict=True)
    ext_bootstrap_dict['F1'].append(clf_report['macro avg']['f1-score'])
    ext_bootstrap_dict['AUPRC'].append(average_precision_score(y_true=y_external.iloc[idx], y_score=prob_pos,average='macro'))    
    ext_bootstrap_dict['AUROC'].append(roc_auc_score(y_true=y_external.iloc[idx],y_score=prob_pos))
    ext_bootstrap_dict['NPV'].append(clf_report['False']['precision'])
    ext_bootstrap_dict['PPV'].append(clf_report['True']['precision'])
    ext_bootstrap_dict['Sensitivity'].append(clf_report['True']['recall'])
    ext_bootstrap_dict['Specificity'].append(clf_report['False']['recall'])

In [ ]:
for key in ext_bootstrap_dict.keys():
    mean = np.round(np.mean(ext_bootstrap_dict[key]),3)
    lower = np.round(np.quantile(ext_bootstrap_dict[key],0.025),3)
    upper = np.round(np.quantile(ext_bootstrap_dict[key],0.975),3)
    print(f'{key}: {mean} (95% CI {lower}-{upper})')

# Feature Importance

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X)

In [ ]:
for column in ['age','lab_crp','lab_urea','lab_ddimer','lab_mcv','lab_platelet','lab_sodium','lab_ast','vitals_spo2_ed_first','lab_creatinine','vitals_hr_ed_first','lab_leukocyte','MALE','lab_neutrophil','lab_glucose','lab_rdw','lab_lymphocyte','lab_aptt','lab_ldh','lab_neutrophil_percentage']:
    # create a dependence plot to show the effect of a single feature across the whole dataset
    shap.dependence_plot(column, shap_values, X)